# Тестовое задание Junior Data Analyst


## Часть 1. Работа с данными

Входные данные для тестового задания можно найти <a href="https://docs.google.com/spreadsheets/d/165sp-lWd1L4qWxggw25DJo_njOCvzdUjAd414NSE8co/edit?usp=sharing">здесь</a>.
Ваша задача - подготовить и обработать исходные данных так, чтобы их можно было использовать во второй части задания.

Требования к выходным данным:
    1) В выходной таблице должны остаться только следующие колонки:
area, cluster, cluster_name, keyword, x, y, count, color, где
 * area - область,
 * cluster - номер кластера,
 * cluster_name - название кластера,
 * keyword - словосочетание,
 * count - показатель,
 * x и y - координаты для диаграммы рассеяния,
 * color - цвет точки на карте для данного словосочетания



In [232]:
import pandas as pd # импортируем библиотеку пандас
import seaborn as sns
import matplotlib.pyplot as plt
import random 
import plotly_express as px
import plotly.io as pio
import plotly.graph_objects as go
from openpyxl import load_workbook


pio.renderers.default='notebook'


In [233]:
# загружаем данные из гугл таблицы
data=pd.read_csv("https://docs.google.com/spreadsheets/d/165sp-lWd1L4qWxggw25DJo_njOCvzdUjAd414NSE8co/gviz/tq?tqx=out:csv")

In [234]:
data.head() # выводим первые 5 строк

,area,cluster,cluster_name,keyword,good (1),count,x,y
0,eligibility,0,Кластер 0,several animated buried,1.0,1260.0,5.772342,12.564796257345005
1,eligibility,0,Кластер 0,singles unusual buyers,1.0,866.0,14.829280,7.8507285727125815
2,eligibility,0,Кластер 0,hawaiian directive,1.0,163.0,11.381856,3.8981370219558604
3,eligibility,0,Кластер 0,dynamics directly,1.0,1146.0,9.980149,6.281427914064545
4,eligibility,1,Кластер 1,decision surgeons montreal,1.0,823.0,3.283940,4.39674063521296


In [235]:
data.shape

(228, 8)

In [236]:
data.dtypes

area             object
cluster           int64
cluster_name     object
keyword          object
good (1)        float64
count           float64
x               float64
y                object
dtype: object

In [237]:
data.y=data.y.astype(float)

ValueError: could not convert string to float: '0x414fe002'

In [238]:
data.loc[data.y=='0x414fe002']

,area,cluster,cluster_name,keyword,good (1),count,x,y
99,worlds,2,Кластер 2,applicants vacuum distance restrictions,1.0,1785.0,11.916011,0x414fe002


In [239]:
data.drop(99,inplace=True)

In [240]:
data.y=data.y.astype(float)

In [241]:
data.drop(columns='good (1)',inplace=True) # удаляем лишнюю колонку

In [242]:
data.head()

,area,cluster,cluster_name,keyword,count,x,y
0,eligibility,0,Кластер 0,several animated buried,1260.0,5.772342,12.564796
1,eligibility,0,Кластер 0,singles unusual buyers,866.0,14.829280,7.850729
2,eligibility,0,Кластер 0,hawaiian directive,163.0,11.381856,3.898137
3,eligibility,0,Кластер 0,dynamics directly,1146.0,9.980149,6.281428
4,eligibility,1,Кластер 1,decision surgeons montreal,823.0,3.283940,4.396741


In [243]:
data.dropna(inplace=True) # удаляем пустые значения

In [244]:
data.shape

(224, 7)

# Работа с колонкой color
 Колонку color нужно добавить самостоятельно - цвета вы можете взять из цветовых палеток Tableu или по своему усмотрению.<br>
    3) Цвет задается каждому словосочетанию согласно следующими правилам:
* внутри одной области цвета словосочетаний в одном кластере должны быть одинаковые, в разных - отличаться (например, у "Кластер 1" все слова будут окрашены в красный, у "Кластер 2" - в зеленый и т.д.)
* цвета кластеров в разных областях могут повторяться
* цвета кластеров в разных областях с разным номером не имеют никакой связи (у одной области [area] слова из "Кластер 1" могут быть красного цвета, в другой области у слов из "Кластер 1" может быть другой цвет)


In [245]:
data["color"]=0

In [246]:
palette=sns.color_palette(palette="tab10").as_hex() # цветовая палитра

In [247]:
palette[0]

'#1f77b4'

### внутри одной области цвета словосочетаний в одном кластере должны быть одинаковые, в разных - отличаться (например, у "Кластер 1" все слова будут окрашены в красный, у "Кластер 2" - в зеленый и т.д.)

In [248]:
areas=data.area.unique() # cписок наших областей

In [249]:
areas

array(['eligibility', 'capability', 'available', 'protein', 'winner',
       'locator', 'worlds', 'ar\\vr', 'twisted', 'lithuania', 'personnel',
       'housewives', 'dialog', 'except', 'greetings'], dtype=object)

In [250]:
clusters=data.cluster.unique() # список кластеров

In [251]:
clusters

array([0, 1, 2, 3])

In [252]:
data

,area,cluster,cluster_name,keyword,count,x,y,color
0,eligibility,0,Кластер 0,several animated buried,1260.0,5.772342,12.564796,0
1,eligibility,0,Кластер 0,singles unusual buyers,866.0,14.829280,7.850729,0
2,eligibility,0,Кластер 0,hawaiian directive,163.0,11.381856,3.898137,0
3,eligibility,0,Кластер 0,dynamics directly,1146.0,9.980149,6.281428,0
4,eligibility,1,Кластер 1,decision surgeons montreal,823.0,3.283940,4.396741,0
...,...,...,...,...,...,...,...,...
223,greetings,2,Кластер 2,disposition layout,279.0,10.971214,4.857810,0
224,greetings,2,Кластер 2,sapphire grounds,335.0,1.160626,3.642820,0
225,greetings,3,Кластер 3,entire ethical speakers,1782.0,7.985910,6.003699,0
226,greetings,3,Кластер 3,courtesy textiles diameter,84.0,0.509490,4.151199,0


In [253]:
for area in areas: # проход по областям
    random.shuffle(palette) # перемешиваем список цветов
    for cluster, color in zip(clusters,palette): # используем zip чтобы для каждого кластера был свой цвет
            data.loc[(data['area']==area) & (data['cluster']==clusters[cluster]),'color']=palette[cluster]
            
                
        
            
            
            



In [254]:
data

,area,cluster,cluster_name,keyword,count,x,y,color
0,eligibility,0,Кластер 0,several animated buried,1260.0,5.772342,12.564796,#2ca02c
1,eligibility,0,Кластер 0,singles unusual buyers,866.0,14.829280,7.850729,#2ca02c
2,eligibility,0,Кластер 0,hawaiian directive,163.0,11.381856,3.898137,#2ca02c
3,eligibility,0,Кластер 0,dynamics directly,1146.0,9.980149,6.281428,#2ca02c
4,eligibility,1,Кластер 1,decision surgeons montreal,823.0,3.283940,4.396741,#bcbd22
...,...,...,...,...,...,...,...,...
223,greetings,2,Кластер 2,disposition layout,279.0,10.971214,4.857810,#ff7f0e
224,greetings,2,Кластер 2,sapphire grounds,335.0,1.160626,3.642820,#ff7f0e
225,greetings,3,Кластер 3,entire ethical speakers,1782.0,7.985910,6.003699,#1f77b4
226,greetings,3,Кластер 3,courtesy textiles diameter,84.0,0.509490,4.151199,#1f77b4


#  Не должно быть дубликатов слов в одной и той же области (area), но словосочетание может повторяться из area в area


In [255]:
#группируем наши данные по областям. Находим количество уникальных значений и общее количество

dublicates=data\
    .groupby('area',as_index=False)\
    .agg({'keyword':[pd.Series.nunique,'count']})

In [256]:
dublicates


area keyword      
                nunique count
0         ar\vr      14    15
1     available      15    15
2    capability      15    15
3        dialog      14    15
4   eligibility      14    16
5        except      15    15
6     greetings      14    15
7    housewives      14    14
8     lithuania      15    15
9       locator      15    15
10    personnel      15    15
11      protein      14    14
12      twisted      14    16
13       winner      15    15
14       worlds      14    14

In [257]:
dublicates['dublicat']=dublicates['keyword']['count']-dublicates['keyword']['nunique'] # подсчитываем количество дубликатом

In [258]:
dublicates.sort_values('dublicat',ascending=False)

area keyword       dublicat
                nunique count         
4   eligibility      14    16        2
12      twisted      14    16        2
0         ar\vr      14    15        1
3        dialog      14    15        1
6     greetings      14    15        1
1     available      15    15        0
2    capability      15    15        0
5        except      15    15        0
7    housewives      14    14        0
8     lithuania      15    15        0
9       locator      15    15        0
10    personnel      15    15        0
11      protein      14    14        0
13       winner      15    15        0
14       worlds      14    14        0

In [259]:
areas_with_dublicates=dublicates.loc[dublicates.dublicat>0].area.to_list() # создаем список областей с дубликатами

In [260]:
areas_with_dublicates 

['ar\\vr', 'dialog', 'eligibility', 'greetings', 'twisted']

In [261]:
data.shape

(224, 8)

In [262]:
for i in areas_with_dublicates: # проходимя по списку и дропаем дабликаты
    data.loc[data.area==i]=data.loc[data.area==i].drop_duplicates(subset='keyword')
    

In [263]:
data.shape

(224, 8)

In [264]:
data.sort_values('area',ascending=True)

,area,cluster,cluster_name,keyword,count,x,y,color
112,ar\vr,1.0,Кластер 1,michael tobacco,1007.0,6.210880,12.721264,#1f77b4
108,ar\vr,0.0,Кластер 0,reservations linking,751.0,10.195602,12.259496,#2ca02c
109,ar\vr,0.0,Кластер 0,committees parallel,173.0,6.735260,3.613983,#2ca02c
110,ar\vr,1.0,Кластер 1,postcards looked republic detector,1397.0,10.474474,6.220012,#1f77b4
111,ar\vr,1.0,Кластер 1,filling volunteers academics,773.0,10.833920,8.652737,#1f77b4
...,...,...,...,...,...,...,...,...
117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [265]:
data.dropna(inplace=True)

In [266]:
# проверка
data\
    .groupby('area',as_index=False)\
    .agg({'keyword':[pd.Series.nunique,'count']})

area keyword      
                nunique count
0         ar\vr      14    14
1     available      15    15
2    capability      15    15
3        dialog      14    14
4   eligibility      14    14
5        except      15    15
6     greetings      14    14
7    housewives      14    14
8     lithuania      15    15
9       locator      15    15
10    personnel      15    15
11      protein      14    14
12      twisted      14    14
13       winner      15    15
14       worlds      14    14

#     4) Колонки должны называться именно так, как указано в п.1


In [267]:
tempdata=data.copy()


In [268]:
tempdata

,area,cluster,cluster_name,keyword,count,x,y,color
0,eligibility,0.0,Кластер 0,several animated buried,1260.0,5.772342,12.564796,#2ca02c
1,eligibility,0.0,Кластер 0,singles unusual buyers,866.0,14.829280,7.850729,#2ca02c
2,eligibility,0.0,Кластер 0,hawaiian directive,163.0,11.381856,3.898137,#2ca02c
3,eligibility,0.0,Кластер 0,dynamics directly,1146.0,9.980149,6.281428,#2ca02c
4,eligibility,1.0,Кластер 1,decision surgeons montreal,823.0,3.283940,4.396741,#bcbd22
...,...,...,...,...,...,...,...,...
223,greetings,2.0,Кластер 2,disposition layout,279.0,10.971214,4.857810,#ff7f0e
224,greetings,2.0,Кластер 2,sapphire grounds,335.0,1.160626,3.642820,#ff7f0e
225,greetings,3.0,Кластер 3,entire ethical speakers,1782.0,7.985910,6.003699,#1f77b4
226,greetings,3.0,Кластер 3,courtesy textiles diameter,84.0,0.509490,4.151199,#1f77b4


In [269]:
tempdata=tempdata.eval('count=@data.x',inplace=False)

In [270]:
tempdata=tempdata.eval('x=@data.y',inplace=False)

In [271]:
tempdata=tempdata.eval("y=@data['count']",inplace=False)

In [272]:
data

,area,cluster,cluster_name,keyword,count,x,y,color
0,eligibility,0.0,Кластер 0,several animated buried,1260.0,5.772342,12.564796,#2ca02c
1,eligibility,0.0,Кластер 0,singles unusual buyers,866.0,14.829280,7.850729,#2ca02c
2,eligibility,0.0,Кластер 0,hawaiian directive,163.0,11.381856,3.898137,#2ca02c
3,eligibility,0.0,Кластер 0,dynamics directly,1146.0,9.980149,6.281428,#2ca02c
4,eligibility,1.0,Кластер 1,decision surgeons montreal,823.0,3.283940,4.396741,#bcbd22
...,...,...,...,...,...,...,...,...
223,greetings,2.0,Кластер 2,disposition layout,279.0,10.971214,4.857810,#ff7f0e
224,greetings,2.0,Кластер 2,sapphire grounds,335.0,1.160626,3.642820,#ff7f0e
225,greetings,3.0,Кластер 3,entire ethical speakers,1782.0,7.985910,6.003699,#1f77b4
226,greetings,3.0,Кластер 3,courtesy textiles diameter,84.0,0.509490,4.151199,#1f77b4


In [273]:
tempdata

,area,cluster,cluster_name,keyword,count,x,y,color
0,eligibility,0.0,Кластер 0,several animated buried,5.772342,12.564796,1260.0,#2ca02c
1,eligibility,0.0,Кластер 0,singles unusual buyers,14.829280,7.850729,866.0,#2ca02c
2,eligibility,0.0,Кластер 0,hawaiian directive,11.381856,3.898137,163.0,#2ca02c
3,eligibility,0.0,Кластер 0,dynamics directly,9.980149,6.281428,1146.0,#2ca02c
4,eligibility,1.0,Кластер 1,decision surgeons montreal,3.283940,4.396741,823.0,#bcbd22
...,...,...,...,...,...,...,...,...
223,greetings,2.0,Кластер 2,disposition layout,10.971214,4.857810,279.0,#ff7f0e
224,greetings,2.0,Кластер 2,sapphire grounds,1.160626,3.642820,335.0,#ff7f0e
225,greetings,3.0,Кластер 3,entire ethical speakers,7.985910,6.003699,1782.0,#1f77b4
226,greetings,3.0,Кластер 3,courtesy textiles diameter,0.509490,4.151199,84.0,#1f77b4


In [274]:
tempdata.rename(columns={'count':'x','x':'y','y':'count'},inplace=True)

In [275]:
data=tempdata

In [276]:
data

,area,cluster,cluster_name,keyword,x,y,count,color
0,eligibility,0.0,Кластер 0,several animated buried,5.772342,12.564796,1260.0,#2ca02c
1,eligibility,0.0,Кластер 0,singles unusual buyers,14.829280,7.850729,866.0,#2ca02c
2,eligibility,0.0,Кластер 0,hawaiian directive,11.381856,3.898137,163.0,#2ca02c
3,eligibility,0.0,Кластер 0,dynamics directly,9.980149,6.281428,1146.0,#2ca02c
4,eligibility,1.0,Кластер 1,decision surgeons montreal,3.283940,4.396741,823.0,#bcbd22
...,...,...,...,...,...,...,...,...
223,greetings,2.0,Кластер 2,disposition layout,10.971214,4.857810,279.0,#ff7f0e
224,greetings,2.0,Кластер 2,sapphire grounds,1.160626,3.642820,335.0,#ff7f0e
225,greetings,3.0,Кластер 3,entire ethical speakers,7.985910,6.003699,1782.0,#1f77b4
226,greetings,3.0,Кластер 3,courtesy textiles diameter,0.509490,4.151199,84.0,#1f77b4


#  Сортировка должна происходить по колонкам area, cluster, cluster_name, count (по count значения сортируются в убывающем порядке, в остальных - по возрастающему).


In [277]:
data=data.sort_values(['area','cluster','cluster_name','count'],ascending=[True,True,True,False])

In [278]:
data.cluster=data.cluster.astype(int)

In [279]:
data

,area,cluster,cluster_name,keyword,x,y,count,color
106,ar\vr,0,Кластер 0,written conflict fabulous,2.991167,7.106799,1443.0,#2ca02c
108,ar\vr,0,Кластер 0,reservations linking,10.195602,12.259496,751.0,#2ca02c
107,ar\vr,0,Кластер 0,interfaces neutral,10.443533,13.809915,586.0,#2ca02c
109,ar\vr,0,Кластер 0,committees parallel,6.735260,3.613983,173.0,#2ca02c
110,ar\vr,1,Кластер 1,postcards looked republic detector,10.474474,6.220012,1397.0,#1f77b4
...,...,...,...,...,...,...,...,...
100,worlds,2,Кластер 2,ringtone parental,11.723895,4.363994,471.0,#2ca02c
102,worlds,2,Кластер 2,recipient traffic,5.593629,0.553368,236.0,#2ca02c
105,worlds,3,Кластер 3,immunology plates,2.407028,7.651527,1653.0,#17becf
103,worlds,3,Кластер 3,holdings herbal,3.986508,10.906340,1476.0,#17becf


In [280]:
data.to_excel("Test_HSE_EX.xlsx",index=False, freeze_panes=(1,0),)

In [281]:
writer = pd.ExcelWriter('Test_HSE_EX.xlsx', engine='openpyxl')
data.to_excel(writer, 'data', index=False)
ws=writer.sheets['data']

ws.auto_filter.ref='A:H'
writer.save()


/tmp/ipykernel_6161/1248926123.py:6: FutureWarning:

save is not part of the public API, usage can give unexpected results and will be removed in a future version



# Часть 2. Построение графиков
На основании обработанных данных постройте по одной диаграмме рассеяния для каждой области (area) (пример внешнего вида см. в приложенном svg-файле).<br>
Требования к визуализации:<br>
* Наличие Footer-подписи на изображении
* Наличие легенды цветов и кластеров
* Перенос слишком длинных словосочетаний (например, слова длиннее 15 символов, можно разбить на "solar\n cell")
* Минимизация наложения (слепливания) подписей к друг на друга (постарайтесь сделать так, чтобы наложение было минимальным)
* Обводка точек.


In [282]:
def improve_text_position(x):
    """ it is more efficient if the x values are sorted """
    # fix indentation 
    positions = ['top center', 'bottom center']  # you can add more: left center ...
    return [positions[i % len(positions)] for i in range(len(x))]

In [283]:
def draw_fig(df,i):
    fig=px.scatter(df,x='x',y='y',
               text='keyword',
               color='cluster_name',
               labels='cluster_name',
               size='count',
               color_discrete_sequence=list(df.color.unique()),
               height=1500,width=1500,
               
              render_mode='svg')
    fig.update_traces(line=dict(color="Black", width=0.5))
    fig.update_layout(xaxis_title=f"{i}'s  clusters",
                  yaxis_title=" ",
                legend_title="Кластеры",

                 font=dict(size=18))
    fig.update_layout({
    'plot_bgcolor': 'white',
    'paper_bgcolor': 'white',
    })
    fig.update_traces(textposition=improve_text_position(worlds['x']))
    fig.write_image(f'{i}.svg')


In [284]:
for i in list(data.area.unique()):
    df=data.loc[data.area==f'{i}']
    df.keyword=df.keyword.apply(lambda x: x if len(x)<16 else x.replace(' ','<br>'))
    draw_fig(df,i)
    
    

/tmp/ipykernel_6161/2143496144.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_6161/2143496144.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_6161/2143496144.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_6161